In [2]:
from langchain.agents import AgentExecutor, create_tool_calling_agent
from langchain_core.runnables import RunnablePassthrough,RunnableLambda, Runnable, RunnableParallel,RunnableConfig,RunnableGenerator
from langchain_core.messages import AIMessage
from dotenv import load_dotenv,find_dotenv
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_openai import ChatOpenAI
from langchain.tools.render import render_text_description
from langchain.tools import tool
from langchain_core.prompts import ChatPromptTemplate,SystemMessagePromptTemplate, HumanMessagePromptTemplate
from langchain_core.output_parsers import JsonOutputParser
from operator import itemgetter
from langchain.embeddings import SentenceTransformerEmbeddings
import json
from langchain_core.output_parsers import StrOutputParser
from langchain_community.vectorstores import FAISS,Chroma
from operator import itemgetter
import time
import grandalf
from typing import Iterator,List,AsyncIterator

In [3]:
load_dotenv(find_dotenv("D:\LLM Courses\Master Langchain Udemy\.env"))

True

In [121]:
llm=ChatGoogleGenerativeAI(model="gemini-1.5-flash")

In [6]:
chain=RunnableParallel(
    extra=RunnablePassthrough.assign(mult=lambda x:x['num']*2),  # Will return a Dictionary within a dictionary
    modified=lambda x : x['num']*2  # Will return a Key-Value Pair                          
)

In [7]:
chain.invoke(input={"num":3})

{'extra': {'num': 3, 'mult': 6}, 'modified': 6}

<h3> Application in Streaming </h3>

In [113]:
vectorStore=FAISS.from_texts(
    texts=["Ritish has a 2.5 month old son !",
           "Rohit is Ritish's Younger Brother",
           "Eklavya is Ritish's Son"
           "Rekha is the name of Ritish's Mother",
           "Anirban is Ritish's Father"],
    embedding=SentenceTransformerEmbeddings())

In [114]:
question={"question":"Who is Anirban's Wife and who is Eklavya's Uncle?"}

In [115]:
prompt=ChatPromptTemplate.from_messages(
    messages=[
            HumanMessagePromptTemplate.from_template
                (
                    template=
                    """
                        Answer the question based only on the following context:
                        {context}
                
                        Question: {question}
                    """
                )
            ],
    
)

In [116]:
retrievalChain=RunnableParallel({"context":vectorStore.as_retriever(k=8),"question":RunnablePassthrough()})|RunnablePassthrough.assign(output=prompt|llm|StrOutputParser())

In [117]:
response=retrievalChain.invoke(input=question['question'])

In [118]:
response

{'context': [Document(page_content="Anirban is Ritish's Father"),
  Document(page_content="Eklavya is Ritish's SonRekha is the name of Ritish's Mother"),
  Document(page_content="Rohit is Ritish's Younger Brother"),
  Document(page_content='Ritish has a 2.5 month old son !')],
 'question': "Who is Anirban's Wife and who is Eklavya's Uncle?",
 'output': 'Based on the provided text, we know:\n\n* **Anirban is Ritish\'s Father** \n* **Eklavya is Ritish\'s Son**\n* **Rohit is Ritish\'s Younger Brother**\n\nWe can deduce the following:\n\n* **Anirban\'s Wife (and therefore Ritish\'s Mother) is Rekha.**  This is stated in the text: "Rekha is the name of Ritish\'s Mother".\n* **Eklavya\'s Uncle is Rohit.**  Since Rohit is Ritish\'s younger brother, he is the uncle of Ritish\'s son, Eklavya. \n\n**Therefore:**\n\n* Anirban\'s Wife is **Rekha**.\n* Eklavya\'s Uncle is **Rohit**. \n'}

In [119]:
print(response['output'])

Based on the provided text, we know:

* **Anirban is Ritish's Father** 
* **Eklavya is Ritish's Son**
* **Rohit is Ritish's Younger Brother**

We can deduce the following:

* **Anirban's Wife (and therefore Ritish's Mother) is Rekha.**  This is stated in the text: "Rekha is the name of Ritish's Mother".
* **Eklavya's Uncle is Rohit.**  Since Rohit is Ritish's younger brother, he is the uncle of Ritish's son, Eklavya. 

**Therefore:**

* Anirban's Wife is **Rekha**.
* Eklavya's Uncle is **Rohit**. 



In [120]:
for chunk in retrievalChain.stream(input=question['question']):
    print(chunk)

{'question': "Who is Anirban's Wife and who is Eklavya's Uncle?"}
{'context': [Document(page_content="Anirban is Ritish's Father"), Document(page_content="Eklavya is Ritish's SonRekha is the name of Ritish's Mother"), Document(page_content="Rohit is Ritish's Younger Brother"), Document(page_content='Ritish has a 2.5 month old son !')]}


Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 2.0 seconds as it raised ResourceExhausted: 429 Resource has been exhausted (e.g. check quota)..
Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 4.0 seconds as it raised ResourceExhausted: 429 Resource has been exhausted (e.g. check quota)..


{'output': 'Based'}
{'output': ' on the provided text, we know:\n\n* **Anirban is Riti'}
{'output': "sh's father.** \n* **Eklavya is Ritish'"}
{'output': "s son.**\n* **Rekha is Ritish's mother.**\n* **Rohit is Ritish's younger brother.**\n\n**Therefore:**\n\n"}
{'output': "* **Anirban's Wife (Ritish's mother) is Rekha.**\n* **Eklavya's Uncle (Riti"}
{'output': "sh's brother) is Rohit.** \n"}


In [ ]:
S